# Traditional Word Embedding

In [2]:
# import packages
from gensim.models import word2vec
from monpa import utils
from tqdm import tqdm
import codecs
import logging
import monpa
import numpy as np
import os
import pandas as pd
import spacy

os.getcwd()

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


'D:\\Windows_Storage\\Storage\\Github\\KnowledgeGraph\\scripts'

# Import Data

In [2]:
textElement = ""

for fileIndex, fileElement in enumerate(tqdm(os.listdir("../data/"))):
    file = codecs.open("../data/" + fileElement, 'r', encoding='utf8', errors='ignore')
    for textIndex, textLines in enumerate(file):
        textElement += textLines
        
    
print("Text Length: ", len(textElement), " \n\n", textElement[-100:])

100%|██████████| 15/15 [00:00<00:00, 306.25it/s]

Text Length:  207715  

 管中流過，凝結下來之蒸

氣附著管外殼上。蒸氣可在管外凝結成一層薄膜後，再排

至儲存槽，或排出後做適當之處置。在接觸式冷凝器中，

則噴灑冷的液體以冷凝廢氣中之揮發性成份。 

5 -30 





In [3]:
# import stopwords list

stopword_list = []

for fileIndex, fileElement in enumerate(os.listdir("./stopwords/")):
    if fileElement[-2:] != "md":
        data_temp = pd.read_csv("./stopwords/" + fileElement, encoding="utf8", sep="@#$%&*")
        stopword_list += data_temp.iloc[:, 0].tolist()
        
stopword_list = list(dict.fromkeys(stopword_list))
        
print(len(stopword_list), "\n\n", stopword_list[-50:])     


2310 

 ['行动', '表明', '表示', '要求', '规定', '觉得', '认为', '认真', '认识', '说明', '转动', '转变', '转贴', '达到', '迅速', '过去', '过来', '运用', '这点', '这种', '这麽', '进入', '进步', '进行', '适应', '适当', '适用', '逐渐', '通常', '造成', '遇到', '遭到', '避免', '那麽', '部分', '采取', '里面', '重大', '重新', '重要', '问题', '防止', '附近', '限制', '随著', '集中', '需要', '高兴', '是不是', '说说']


<ipython-input-3-caa2e0aa211a>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data_temp = pd.read_csv("./stopwords/" + fileElement, encoding="utf8", sep="@#$%&*")


# Create User-Defined Dict by entities

In [4]:
data_dict = pd.read_csv("../results/210330_result/210330_dataset_entity_result_MONPA.csv", sep=",", encoding="utf8")

entity_list = data_dict.iloc[:, 0].tolist()

# remove blank inside entity element
for elementIndex, element in enumerate(entity_list):
    entity_list[elementIndex] = entity_list[elementIndex].replace(" ", "")
    entity_list[elementIndex] = entity_list[elementIndex].replace("  ", "")
    entity_list[elementIndex] = entity_list[elementIndex].replace("\"", "")


data_dict_output = pd.DataFrame({
    "0": entity_list,
    "1": [int(1000000000 * float(str(data_dict.iloc[i, -1])[:-1])) for i in range(len(entity_list))],
    "2": ["UserEntity" for i in range(len(entity_list))],
})

data_dict_output.to_csv("./dicts/monpa_entity_dict.txt", encoding="utf8", sep=" ", index=None, header=None)

# Segment data 

In [5]:
# read in entity dictionary

monpa.load_userdict("./dicts/monpa_entity_dict.txt")

In [6]:
# divide sentences and segment by monpa
textList = utils.short_sentence(textElement)

cutResultList = []

for textIndex, textElement in enumerate(tqdm(textList)):
    cutResult = monpa.cut(textElement)
    for elementIndex, element in enumerate(cutResult):
        if element in stopword_list:
            cutResult[elementIndex] = ""
    cutResultList.append(cutResult)

print(cutResultList[:5])

100%|██████████| 1332/1332 [08:29<00:00,  2.61it/s]

[['安森', '美', '半導體', '', 'O', 'N Se', 'm', 'icon', 'd', 'uctor', '', '', '', '天', '發', '表', '新', '', 'R', 'D', 'M', '系列', '矽', '光', '電', '倍', '增', '', ' ', '', 'Si', 'P', 'M', '', '陣列', '', '將', '光', '學', '雷', '達', '', 'Li', 'D', 'A', 'R', '', ' 感', '測', '器', '', '力', '擴展到其', '廣', '泛', '', '智慧感', '測', '方案', '陣容', '', 'A', 'rray', 'R', 'D', 'M', '', '', '', '12', 'A', '20', '', 'Q', 'F', 'N', '', '市場', '', '首款', '符合', '車規', '', 'Si', 'P', 'M', '產品', '', '', '滿足', '汽車', '產業', '', '', '', '領', '域', 'Li', 'D', 'A', 'R', '應用', '中', '不斷', '增', '長', '', '需求', ''], ['A', 'rray', 'R', 'D', 'M', '', '', '', '12', 'A', '20', '', 'Q', 'F', 'N', '', '單', '片', '', '', '12', 'Si', 'P', 'M', '', '素', '陣列', '', '基', '於', '安森', '美', '半導體', '領', '先', '市場', '', 'R', 'D', 'M', '製程', '', '', '實', '現', '對', '近', '紅', '外', '', 'NI', 'R', '', '光', '', '高', '靈敏', '度', '', '從', '', '', '90', '', '奈米', '', 'n', 'm', '', '處', '達到', '領', '先', '業界', '', '18', '', '', '', '', '光', '子', '偵', '測', '效', '率', ' ', '', 'P'

In [7]:
# create data for word2vec training

lineList = []

for sentencesIndex, sentencesElementList in enumerate(tqdm(cutResultList)):
    tokenElement = ""
    for elementIndex, element in enumerate(sentencesElementList):
        tokenElement += (element + " ")
    lineList.append(tokenElement)
    
data_output_word2vec = pd.DataFrame({
    "Token Sentence":lineList
})

data_output_word2vec.to_csv("../results/210330_result/210330_token_sentences_for_word2vec.csv", encoding="utf8", sep=" ", header=None, index=None)

100%|██████████| 1332/1332 [00:00<00:00, 16049.77it/s]


# Word2Vec Model Training
* Reference: https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py
* Reference: https://radimrehurek.com/gensim/auto_examples/tutorials/run_annoy.html

In [2]:
# import packages
from gensim.models import word2vec
from monpa import utils
from tqdm import tqdm
import codecs
import logging
import monpa
import numpy as np
import os
import pandas as pd
import spacy

os.getcwd()

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


'D:\\Windows_Storage\\Storage\\Github\\KnowledgeGraph\\scripts'

In [4]:
sentences = word2vec.LineSentence("../results/210330_result/210330_token_sentences_for_word2vec.csv")
model = word2vec.Word2Vec(sentences, size=240)

model.save("../models/210330_word2vec.model")

In [3]:
model = word2vec.Word2Vec.load("../models/210330_word2vec.model")

print(model.wv.similarity("晶圓", "價值"))

0.9057946


In [ ]:
print(model.wv.most_similar(["晶圓", "矽"]))
